In [1]:
# import os
# os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
# os.environ["RUNFILES_DIR"] = "/Library/Frameworks/Python.framework/Versions/3.8/share/plaidml"
# os.environ["PLAIDML_NATIVE_PATH"] = "/Library/Frameworks/Python.framework/Versions/3.8/lib/libplaidml.dylib"

In [2]:
import numpy as np
import pickle
import cv2

from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from PIL import Image

In [3]:
EPOCHS = 15
INIT_LR = 1e-3
BS = 32
default_image_size = tuple((256, 256))
image_size = 0
directory_root = './train'
width=256
height=256
depth=3

In [4]:
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            image = cv2.resize(image, default_image_size)   
            return img_to_array(image)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

In [9]:
image_list, label_list = [], []
try:
    print("[INFO] Loading images ...")
    root_dir = listdir(directory_root)
    for directory in root_dir :
        # remove .DS_Store from list
        if directory == ".DS_Store" :
            root_dir.remove(directory)
        

    for plant_disease_folder in root_dir:
        print(f"[INFO] Processing {plant_disease_folder} ...")
        plant_disease_image_list = listdir(f"{directory_root}/{plant_disease_folder}/")

        for single_plant_disease_image in plant_disease_image_list :
            if single_plant_disease_image == ".DS_Store" :
                plant_disease_image_list.remove(single_plant_disease_image)

        for image in plant_disease_image_list[:1500]:
            image_directory = f"{directory_root}/{plant_disease_folder}/{image}"
            if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True:
                image_list.append(convert_image_to_array(image_directory))
                label_list.append(plant_disease_folder)
    print("[INFO] Image loading completed")  
except Exception as e:
    print(f"Error : {e}")

[INFO] Loading images ...
[INFO] Processing Tomato___Bacterial_spot ...
[INFO] Processing Tomato___Early_blight ...
[INFO] Processing Tomato___healthy ...
[INFO] Processing Tomato___Late_blight ...
[INFO] Processing Tomato___Leaf_Mold ...
[INFO] Processing Tomato___Septoria_leaf_spot ...
[INFO] Processing Tomato___Spider_mites Two-spotted_spider_mite ...
[INFO] Processing Tomato___Target_Spot ...
[INFO] Processing Tomato___Tomato_mosaic_virus ...
[INFO] Processing Tomato___Tomato_Yellow_Leaf_Curl_Virus ...
[INFO] Image loading completed


In [10]:
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)
pickle.dump(label_binarizer,open('label_transform.pkl', 'wb'))
n_classes = len(label_binarizer.classes_)
np_image_list = np.array(image_list) / 255.0
print("[INFO] Spliting data to train, test")
x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_labels, test_size=0.2, random_state = 42) 

[INFO] Spliting data to train, test


In [11]:
model = Sequential()
inputShape = (height, width, depth)
chanDim = -1
if K.image_data_format() == "channels_first":
    inputShape = (depth, height, width)
    chanDim = 1
model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
# fully-connected layer
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
# softmax classifier
model.add(Dense(n_classes))
model.add(Activation("sigmoid"))

In [13]:
from numpy.random import seed
import tensorflow as tf
seed(1)
import tensorflow
tensorflow.random.set_seed(1)
from keras.callbacks import ModelCheckpoint


from keras.callbacks import EarlyStopping, ModelCheckpoint

early_stop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=10, mode='min', verbose=1)
checkpoint = ModelCheckpoint('model_best_weights.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', period=1)

opt= Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
# distribution
model.compile(loss='binary_crossentropy', optimizer=opt,metrics=["accuracy"])
# train the network
print("[INFO] training network...")

aug = ImageDataGenerator(
    rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, 
    zoom_range=0.2,horizontal_flip=True, 
    fill_mode="nearest")
history = model.fit(
    x_train,y_train,
    validation_data=(x_test, y_test),callbacks=[early_stop,checkpoint],
    steps_per_epoch=len(x_train) // BS,
    epochs=EPOCHS, verbose=1)

[INFO] training network...
Epoch 1/15
375/375 [==============================] - ETA: 0s - loss: 0.2849 - accuracy: 0.6112
Epoch 00001: val_loss improved from inf to 1.19297, saving model to model_best_weights.h5
375/375 [==============================] - 74s 198ms/step - loss: 0.2849 - accuracy: 0.6112 - val_loss: 1.1930 - val_accuracy: 0.1440
Epoch 2/15
375/375 [==============================] - ETA: 0s - loss: 0.1236 - accuracy: 0.7978
Epoch 00002: val_loss improved from 1.19297 to 0.09999, saving model to model_best_weights.h5
375/375 [==============================] - 70s 188ms/step - loss: 0.1236 - accuracy: 0.7978 - val_loss: 0.1000 - val_accuracy: 0.8267
Epoch 3/15
375/375 [==============================] - ETA: 0s - loss: 0.0991 - accuracy: 0.8432
Epoch 00003: val_loss did not improve from 0.09999
375/375 [==============================] - 68s 181ms/step - loss: 0.0991 - accuracy: 0.8432 - val_loss: 0.6785 - val_accuracy: 0.4220
Epoch 4/15
375/375 [============================

In [14]:
import tensorflow as tf
model1=tf.keras.models.load_model('./model_best_weights.h5')

In [ ]:
# Outlier detection 
def quantify_image(image, bins=(4, 6, 3)):
    # compute a 3D color histogram over the image and normalize it
    hist = cv2.calcHist([image], [0, 1, 2], None, bins,
    [0, 180, 0, 256, 0, 256])
    hist = cv2.normalize(hist, hist).flatten()
    # return the histogram
    return hist

def load_dataset(datasetPath, bins):
    # grab the paths to all images in our dataset directory, then
    # initialize our lists of images
    imagePaths = list(paths.list_images(datasetPath))
    data = []
    # loop over the image paths
    for imagePath in imagePaths:
        # load the image and convert it to the HSV color space
        image = Image.open(imagePath)
        image = np.array(image)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
        # quantify the image and update the data list
        features = quantify_image(image, bins)
        data.append(features)
    # return our data list as a NumPy array
    return np.array(data)

def anomoly_detect(image):
    model_anomoly=pickle.load(open('model.hdf5','rb'))
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    features = quantify_image(hsv, bins=(4, 6, 3))
    preds = model_anomoly.predict([features])[0]
    return preds


data=load_dataset('./data/train_data/',bins=(4,6,3))
model = IsolationForest(n_estimators=50, contamination=0.01,random_state=42)
model.fit(data)
pickle.dump(model, open('model.hdf5', 'wb'))


In [16]:
directory_root = './validation'
image_list, label_list = [], []
try:
    print("[INFO] Loading images ...")
    root_dir = listdir(directory_root)
    for directory in root_dir :
        # remove .DS_Store from list
        if directory == ".DS_Store" :
            root_dir.remove(directory)

#     for plant_folder in root_dir :
#         plant_disease_folder_list = listdir(f"{directory_root}/{plant_folder}")
        

    for plant_disease_folder in root_dir:
        print(f"[INFO] Processing {plant_disease_folder} ...")
        plant_disease_image_list = listdir(f"{directory_root}/{plant_disease_folder}/")

        for single_plant_disease_image in plant_disease_image_list :
            if single_plant_disease_image == ".DS_Store" :
                plant_disease_image_list.remove(single_plant_disease_image)

        for image in plant_disease_image_list[:500]:
            image_directory = f"{directory_root}/{plant_disease_folder}/{image}"
            if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True:
                image_list.append(convert_image_to_array(image_directory))
                label_list.append(plant_disease_folder)
    print("[INFO] Image loading completed")  
except Exception as e:
    print(f"Error : {e}")

[INFO] Loading images ...
[INFO] Processing Tomato___Bacterial_spot ...
[INFO] Processing Tomato___Early_blight ...
[INFO] Processing Tomato___healthy ...
[INFO] Processing Tomato___Late_blight ...
[INFO] Processing Tomato___Leaf_Mold ...
[INFO] Processing Tomato___Septoria_leaf_spot ...
[INFO] Processing Tomato___Spider_mites Two-spotted_spider_mite ...
[INFO] Processing Tomato___Target_Spot ...
[INFO] Processing Tomato___Tomato_mosaic_virus ...
[INFO] Processing Tomato___Tomato_Yellow_Leaf_Curl_Virus ...
[INFO] Image loading completed


In [17]:
x_test=np.array(image_list)/255 

In [18]:
y_test=label_binarizer.transform(label_list)

In [19]:
print("[INFO] Calculating model accuracy")
scores = model1.evaluate(x_test,y_test)
print(f"Test Accuracy: {scores[1]*100}")

[INFO] Calculating model accuracy
144/144 [==============================] - 7s 48ms/step - loss: 0.0549 - accuracy: 0.9280
Test Accuracy: 92.80261993408203


In [20]:
x_pre=model1.predict(x_test,verbose=1)

144/144 [==============================] - 6s 43ms/step


In [21]:
from sklearn.metrics import classification_report
print(classification_report(np.argmax(y_test,axis=-1),np.argmax(x_pre, axis=-1)))

              precision    recall  f1-score   support

           0       0.97      0.92      0.94       425
           1       0.92      0.88      0.90       480
           2       0.87      0.93      0.90       463
           3       0.98      0.94      0.96       470
           4       0.90      0.90      0.90       436
           5       0.90      0.93      0.91       435
           6       0.95      0.82      0.88       457
           7       0.94      0.97      0.95       490
           8       0.92      0.99      0.95       448
           9       0.96      0.99      0.98       481

    accuracy                           0.93      4585
   macro avg       0.93      0.93      0.93      4585
weighted avg       0.93      0.93      0.93      4585

